In [1]:
from artistplaylist import ArtistPlaylist
import event_scraper
import pprint

events = event_scraper.get_events(10, 100)
print("Found {0} events".format(len(events)))

Found 100 events


In [2]:
for ev in events[-15:]:
    print(' -- '.join([str(ev['date']), ev['artist']]))

2017-11-04 -- Dharma Kings
2017-11-04 -- Dongle
2017-11-04 -- Wizard U.
2017-11-04 -- DOJA CAT
2017-11-04 -- Lug
2017-11-04 -- Death Of A Dream
2017-11-04 -- Shadow Spectrum
2017-11-04 -- Nocebos
2017-11-04 -- Fear A New Day
2017-11-04 -- Jim Gaffigan
2017-11-04 -- Town Mountain
2017-11-04 -- Beat Root Revival
2017-11-04 -- Julie Nolen
2017-11-04 -- Danny Click Full Band Shows
2017-11-04 -- LezzDance


In [4]:
ap = ArtistPlaylist()

MAX_SONG_COUNT = 25
song_ids = []
for ev in events:
    songs = ap.songs_from_artist(ev['artist'])
    result = None
    sid = None
    if songs:
        sid = songs[0]['storeId']
        song_ids.append(sid)
        result = songs[0].get('genre', None)
    print(' -- '.join([ev['artist'], str(sid) , str(result)]))
    if len(song_ids) > MAX_SONG_COUNT: break

INFO:gmusicapi.Mobileclient2:initialized
INFO:gmusicapi.Mobileclient2:logged out
INFO:gmusicapi.Mobileclient2:authenticated
INFO:gmusicapi.Mobileclient2:subscribed


Autograf and Ayokay -- None -- None
Angelo Moore and the Brand New Step -- None -- None
Charlotte DJ Set -- None -- None
Lil Mike & Funny Bone, Lil Mike & FunnyBone -- None -- None
Carter Arrington -- None -- None
Young and Rusty -- Tp5gsmrdmdooy4uhpaqayn3rkne -- Country
Tiarra Girls, Grupo Fantasma, Bombasta and 3 more... -- None -- None
Teddy Long Music -- None -- None
Werefoot, Ayasoltec, Toulca and 6 more... -- None -- None
Oscar Ornelas -- None -- None
Urban Pioneers -- Tvues3s7oe7or6ilv5xmjwvlwxy -- Classic Country
Tara Williamson Music -- None -- None
Riders Against the Storm -- T2fatrs7urfxme5536txhkis2bq -- Hip-Hop/Rap
Bamako Airlines -- T3i3wqj3swp76tvmsnnwtb3t52a -- World
Abou Sylla -- Tckaex4ma5tztzekeicsfqbb4za -- World
Roots Ridim -- None -- None
DJ Truth -- None -- None
Emman LeGrand -- T2uitmshamhr3fugknjpi5wxqtq -- World
the African Rhythmix -- None -- None


ERROR:gmusicapi.protocol.shared:the response format for Search was not recognized.

Value {'kind': 'sj#radioStation', 'name': 'Blogged 50', 'description': 'A curated selection of the best songs trending on the Internet right now.', 'seed': {'kind': 'sj#radioSeed', 'curatedStationId': 'Lj7guwddznce3uza7ox5w5ahruy', 'seedType': '9'}, 'stationSeeds': [{'kind': 'sj#radioSeed', 'curatedStationId': 'Lj7guwddznce3uza7ox5w5ahruy', 'seedType': '9'}], 'imageUrls': [{'kind': 'sj#imageRef', 'url': 'http://lh3.googleusercontent.com/JDYALzRXMASteXamftOuzqyqGFonNMKee3oKUS3DFuBKmBP05hPG1ANobJRoweIilTn0nEzIUlc', 'aspectRatio': '1', 'autogen': False}, {'kind': 'sj#imageRef', 'url': 'http://lh3.googleusercontent.com/OvzDMbtDhLUnJbPguaM5f04a8eqRX5V5nfR_eF4FVYfV2zvXJIpmAYJvTxCNzwu4_FzVJMQg2w', 'aspectRatio': '1', 'autogen': False}, {'kind': 'sj#imageRef', 'url': 'http://lh3.googleusercontent.com/4FkSx54p1zKKoXhQ6riGcQNMOYOmpQmYkBfXfKPViGW59aomVI6OwqXO3JGvY9ZdSzQNAWpPz38', 'aspectRatio': '1', 'autogen': Fal

Better Now -- None -- None
El Tule -- Ti4o7anqhsofuydnmcmgpd3vqsa -- Latin
Mike and the Moonpies -- Ty7igxuuvqnx7wzloife6obefdi -- None
Hoodygoode -- Thh6cliiv43l74chik7j3gmgdga -- Rock
Bobby Whitlock -- Tffziai3as3msoni6rp2gqmmqg4 -- Rock
Coco Carmel -- Tmtqnx4p6sxqxoxdvkvqfdsogsy -- '70s Rock
WC Clark -- None -- None
Steven Ray Will -- Tv4eiby7qa7t2qrvgydpd3a4vdi -- Rock
Rock Bottom String Band, Black Eyed Vermillion, Urban Pioneers and 11 more... -- None -- None
Redd Volkaert -- Tultwvrqhtqekwqp6olv6ylbdvq -- Country
Igor -- Twsa6wtitsaq2waoeivfaa4sgwe -- Classical
DJ Buck Rodgers -- None -- None
Savannah Red Music -- None -- None
Scorpio Rising -- T2bbas3vf7viwujm5u265iqoh7u -- Dance/Electronic
Go Fever -- Tntelpu4vkegdnnanz222vfjece -- Surf
MIDDLESPOON, the Deer, Shaws of Awe and 1 more... -- None -- None
Rock Bottom String Band, the Deer, Fire In The Pines and 3 more... -- None -- None
The Latin Dukes -- None -- None
raccoon brothers, raccoon brothers -- None -- None
One Eye Open

In [5]:
import ipywidgets as widgets
from IPython.display import display
button = widgets.Button(description="Publish Playlist")
display(button)

PUBLISH_PUBLICALLY = True

def on_button_clicked(b):
    ap.create_playlist(song_ids, 'Austin Live Artists - V0.1', public=PUBLISH_PUBLICALLY)
    print("Published playlist with {} songs".format(len(song_ids)))

button.on_click(on_button_clicked)

button_delete = widgets.Button(description="Delete Playlist")
display(button_delete)

def on_delete_clicked(b):
    ap.delete_playlist()
    print("Deleted playlist")

button_delete.on_click(on_delete_clicked)

Button(description='Publish Playlist', style=ButtonStyle())

Button(description='Delete Playlist', style=ButtonStyle())

Published playlist with 26 songs
